In [2]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

In [3]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

In [3]:
conn.execute("DROP TABLE IF EXISTS especies_nativas_endemicas")
conn.execute("""
    CREATE TABLE especies_nativas_endemicas AS
    SELECT * FROM especies_nativas
    UNION ALL
    SELECT * FROM especies_endemicas
""")

In [5]:
print(conn.execute("DESCRIBE gbif").df()["column_name"].tolist())

['key', 'datasetKey', 'publishingOrgKey', 'networkKeys', 'installationKey', 'hostingOrganizationKey', 'publishingCountry', 'protocol', 'lastCrawled', 'lastParsed', 'crawlId', 'basisOfRecord', 'occurrenceStatus', 'taxonKey', 'kingdomKey', 'phylumKey', 'classKey', 'orderKey', 'familyKey', 'genusKey', 'speciesKey', 'acceptedTaxonKey', 'scientificName', 'scientificNameAuthorship', 'acceptedScientificName', 'kingdom', 'phylum', 'order', 'family', 'genus', 'species', 'genericName', 'specificEpithet', 'taxonRank', 'taxonomicStatus', 'iucnRedListCategory', 'decimalLatitude', 'decimalLongitude', 'continent', 'stateProvince', 'year', 'month', 'day', 'eventDate', 'startDayOfYear', 'endDayOfYear', 'issues', 'lastInterpreted', 'license', 'isSequenced', 'identifiers', 'media', 'facts', 'relations', 'institutionKey', 'collectionKey', 'isInCluster', 'recordedBy', 'identifiedBy', 'dnaSequenceID', 'geodeticDatum', 'class', 'countryCode', 'recordedByIDs', 'identifiedByIDs', 'gbifRegion', 'country', 'publ

In [6]:
conn.execute("DROP TABLE IF EXISTS gbif_nativas_endemicas")
conn.execute("""
    CREATE TABLE gbif_nativas_endemicas AS
    SELECT *
    FROM gbif
    WHERE species IN (
        SELECT DISTINCT(scientificName_y)
        FROM especies_nativas_endemicas
    )
""").df()

,Count
0,22378


In [9]:
df = conn.execute("""
    SELECT species, year, namePublishedInYear
    FROM gbif_nativas_endemicas;
""").df()
df.head()

,species,year,namePublishedInYear
0,Piper melanocladum,1988.0,None
1,Piper hispidum,1988.0,None
2,Piper hispidum,1988.0,None
3,Piper cernuum,1988.0,None
4,Piper aduncum,1988.0,None


In [5]:
df["enumRedListCategory"] = pd.factorize(df['redlistCategory'])[0]
df.head()

,scientificName_y,redlistCategory,region_count,enumRedListCategory
0,Piper amalago,Least Concern,35,0
1,Piper aduncum,Least Concern,32,0
2,Piper hispidum,Least Concern,27,0
3,Piper tuberculatum,Least Concern,26,0
4,Piper dilatatum,Least Concern,25,0


In [ ]:
df.to_csv("regression.csv", sep=";", quoting=2, index=False, encoding="utf-8")

In [7]:
conn.close()